In [1]:
# -*- coding: utf-8 -*-

import os
import httplib2
import sys
import google.oauth2.credentials
import google_auth_oauthlib.flow

from googleapiclient.discovery import build, build_from_document
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from apiclient.discovery import build
from oauth2client.tools import argparser, run_flow
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage


import pandas as pd
import downloadPermission

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import re

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [3]:
dbname = 'youtubeSpon'
username = 'april' # change this to your username

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
con = None
con = psycopg2.connect(database = dbname, user = username)
# query:
sql_query = """
SELECT * FROM "youtubeEmmy" WHERE sponsored <> 'None';
"""
youtube_data_from_sql = pd.read_sql_query(sql_query,con)
youtube_data_from_sql.tail()

,index,channel,title,video_id,description,captions,sponsored,fullCaptions
253,810,emmymadeinjapan,WINNERS Curry Set Giveaway,CQTp_Hh8qH4,Announcing the winners of the Popin Cookin' Ka...,"1\n00:00:06,680 --> 00:00:11,519\nhi everyone ...",no,grins uglies hides and we welcome back to anot...
254,811,emmymadeinjapan,PEZ giveaway WINNERS,stpWXStiw4k,Watch to find out if you're one of the lucky w...,"1\n00:00:08,540 --> 00:00:13,200\nhello everyb...",no,hello everybody I'm back again and it's it's s...
255,814,emmymadeinjapan,PEZ Whatcha Eating #82,yJfm6wFnr2s,[GIVEAWAY CLOSED] Eating old fashioned PEZ can...,"1\n00:00:08,470 --> 00:00:15,980\nhello everyb...",no,greetings my lovelies hi and welcome back to a...
256,815,emmymadeinjapan,Gummy Burger Set - Lunch Bag: Whatcha Eating? #81,9cbb_kdUspU,[GIVEAWAY CLOSED] A miniature gummy version of...,"1\n00:00:07,910 --> 00:00:14,639\nhi everybody...",no,everybody it's a me and I'm back and I'm still...
257,836,emmymadeinjapan,Giveaway Winners - Chocolat de Tomato,R-VvqR0EFNk,Wanna try Japanese tomato chocolate for yourse...,"1\n00:00:07,280 --> 00:00:12,870\nhello you tw...",no,frooty frooty frooty frooty frooty frooty brin...


In [24]:
import random
import nltk

In [62]:
sample_size = 15
not_spon = [index for index, item in enumerate(youtube_data_from_sql.sponsored) if item == 'no']
not_spon_sample = random.sample(not_spon,sample_size)
spon = [index for index, item in enumerate(youtube_data_from_sql.sponsored) if item == 'yes']
spon_sample = random.sample(spon,sample_size)

In [63]:
train = youtube_data_from_sql.iloc[not_spon_sample + spon_sample, :]
train

,index,channel,title,video_id,description,captions,sponsored,fullCaptions
132,150,emmymadeinjapan,LITHUANIAN Military Ration Taste Test | Lithua...,nPiuSUgpJ0Q,Find out what's in a Lithuanian military ratio...,"1\n00:00:05,270 --> 00:00:10,320\nwe changed m...",no,it's Emmy and I'm back again with another how ...
256,815,emmymadeinjapan,Gummy Burger Set - Lunch Bag: Whatcha Eating? #81,9cbb_kdUspU,[GIVEAWAY CLOSED] A miniature gummy version of...,"1\n00:00:07,910 --> 00:00:14,639\nhi everybody...",no,everybody it's a me and I'm back and I'm still...
134,152,emmymadeinjapan,Pop-Up HOT DOG TOASTER Test | Does It Work?,F5gjlXa8GZ8,"The Pop-up Hot Dog Toaster, it's red and can h...","1\n00:00:00,060 --> 00:00:08,099\n[Music]\n\n2...",no,screams my lovelies Hyatts and we welcome back...
81,98,emmymadeinjapan,BAO Chinese Pastries Taste Test,VOQ6iTdZnqE,Baos are delightful Chinese pastries that make...,"1\n00:00:00,040 --> 00:00:07,410\n[Music]\n\n2...",no,[Music] greetings my beautiful lovelies hides ...
171,189,emmymadeinjapan,GERMAN 24-Hour Ration TASTE TEST | Einmannpack...,qz6BRSNXGtY,Having my first taste of a German 24-hour mili...,"1\n00:00:04,400 --> 00:00:08,519\ngreetings my...",no,today I meet in another country I've received ...
168,186,emmymadeinjapan,CACTUS PEAR | Prickly Pear Taste Test - FRUITY...,cc9lAlwqlFc,"It bleeds red and grows in the desert, it's ca...","1\n00:00:00,030 --> 00:00:06,210\nfrooty froot...",no,greetings my lovelies hi it's Emmy welcome bac...
106,124,emmymadeinjapan,Trader Joe's KNOCK-OFF Candy Bar TASTE TEST | ...,MK_wNXdIDd8,Trader Joe's has a couple of candy bars that s...,"1\n00:00:01,740 --> 00:00:04,930\n[Music]\n\n2...",no,[A Mission music] Greetings my beautiful lovel...
154,172,emmymadeinjapan,"CHEETOS COOKIE Recipe | 3 Ways - Flamin' Hot, ...",1N0R2WsHUk4,Here's how to make Cheetos Cookies in three fl...,"1\n00:00:05,820 --> 00:00:07,760\nGreeting lov...",no,[Music] greetings lovelies hides and we welcom...
5,6,emmymadeinjapan,WASABI MAYO Noodle Challenge | Samyang WasaMayo,oW-KrXHKL4k,Samyang has added another instant noodle flavo...,"1\n00:00:00,080 --> 00:00:07,799\n[Music]\n\n2...",no,greetings up loose hi it's Amy welcome back to...
26,28,emmymadeinjapan,MEATLESS TURKEY | made with skewered vegan hot...,WD3UeGN0zMw,Have you heard of the carrot dog? The marinat...,"1\n00:00:00,750 --> 00:00:02,849\n[Music]\n\n2...",no,welcome to Tacoma today an annual festival hel...


,index,channel,title,video_id,description,captions,sponsored,fullCaptions
202,221,emmymadeinjapan,Gudetama Pudding Kit - Whatcha Eating?,ifPaYqXSo3k,It's everyone's favorite sad egg yolk Gudetama...,"1\n00:00:00,350 --> 00:00:03,540\n[Music]\n\n2...",no,greetings my beautiful lovelies heights and we...
206,226,emmymadeinjapan,Chicken FEET Taste Test & Dim Sum Recipe - You...,38xQGhevJ6M,Here's how to make classic dim sum-style chick...,"1\n00:00:00,320 --> 00:00:08,970\n[Music]\n\n2...",no,greetings always hi it's ma I am back to share...
84,101,emmymadeinjapan,KOREAN CHEETOS Taste Test,yORcJPXKWp4,Visit https://squarespace.com/emmy and enter t...,"1\n00:00:00,090 --> 00:00:07,740\n[Music]\n\n2...",yes,hi everyone I'm back and today I am back with ...
30,33,emmymadeinjapan,Emmy REACTS To Her 1st VIDEO | My 7-year YouTu...,BztZKL8BScY,This video IS sponsored by Best Fiends. Click...,"1\n00:00:00,000 --> 00:00:02,540\n[Music]\n\n2...",yes,greetings lovely people it's Emily and I am ba...


In [142]:
from nltk.tokenize import word_tokenize # or use some other tokenizer
from nltk.corpus import stopwords
import string
stop = stopwords.words('english') + list(string.punctuation)

In [157]:
train_tokens = []
for sent in train.fullCaptions:
    train_tokens.append([i for i in word_tokenize(sent.lower()) if i not in stop])

In [161]:
words = [y for x in train_tokens for y in x]

In [162]:
all_words = nltk.FreqDist(w.lower() for w in words)
word_features = list(all_words)

def document_features(document):
    document_words = document.split()
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [163]:
feature_set = []
for i in range(len(train)):
    feature_set.append((document_features(train.fullCaptions.iloc[i]), 
           train.sponsored.iloc[i]))

In [164]:
classifier = nltk.NaiveBayesClassifier.train(feature_set)

In [165]:
classifier.show_most_informative_features(20)

Most Informative Features
          contains(mint) = True              yes : no     =      5.0 : 1.0
           contains(wow) = True              yes : no     =      4.3 : 1.0
         contains(thick) = True              yes : no     =      4.3 : 1.0
          contains(many) = True              yes : no     =      4.3 : 1.0
          contains(bits) = True               no : yes    =      4.3 : 1.0
          contains(soft) = True              yes : no     =      3.8 : 1.0
         contains(great) = False              no : yes    =      3.7 : 1.0
        contains(either) = True               no : yes    =      3.7 : 1.0
   contains(information) = True              yes : no     =      3.7 : 1.0
        contains(missed) = True              yes : no     =      3.7 : 1.0
        contains(lovely) = True              yes : no     =      3.7 : 1.0
          contains(whoa) = True              yes : no     =      3.7 : 1.0
        contains(overly) = True              yes : no     =      3.7 : 1.0

In [166]:
train.description.iloc[['mint'in cap for cap in train.captions]]

171    Having my first taste of a German 24-hour mili...
188    A big thank you to Progresso for sponsoring th...
101    Visit https://squarespace.com/emmy and enter t...
Name: description, dtype: object

In [170]:
spon_test = list(set(spon) - set(spon_sample))
not_spon_test =  random.sample(list(set(not_spon) - set(not_spon_sample)),len(spon_test))

test = youtube_data_from_sql.iloc[not_spon_test + spon_test, :]

test_feature_set = []
for i in range(len(test)):
    test_feature_set.append((document_features(test.fullCaptions.iloc[i]), 
           test.sponsored.iloc[i]))
    
    
nltk.classify.accuracy(classifier, test_feature_set)

0.25

In [171]:
i=0
(classifier.classify(document_features(test.fullCaptions.iloc[i])), test.sponsored.iloc[i])

('yes', 'no')